In [1]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from dataset import NHANESDataset
import numpy as np
import pandas as pd

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# NHANES

In [2]:
data = NHANESDataset()

/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/dataset.py:211: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.data.replace({False: 0, True: 1}, inplace=True)


In [3]:
d = data.get_data()
d

array([[  0.        ,  51.        ,   3.        , ..., 110.        ,
         40.        ,  25.40680287],
       [  0.        ,  41.        ,   2.        , ..., 136.        ,
         54.        ,  24.58833109],
       [  1.        ,  31.        ,   2.        , ..., 110.        ,
         24.        ,  23.75650236],
       ...,
       [  1.        ,  47.        ,   2.        , ..., 108.        ,
         28.        ,  17.9151988 ],
       [  1.        ,  61.        ,   8.        , ..., 180.        ,
         68.        ,  27.33162171],
       [  0.        ,  29.        ,   3.        , ..., 152.        ,
         60.        ,  33.16143518]])

In [4]:
X_train, X_test, y_train, y_test = data.get_train_test_xgboost()

In [5]:
y_train

,Survival_label_lower_bound,Survival_label_upper_bound
7556,13.750000,13.750000
819,11.500000,11.500000
12960,17.833333,inf
4609,12.083333,12.083333
5028,20.083333,inf
...,...,...
5191,2.500000,2.500000
13418,0.833333,0.833333
5390,20.000000,inf
860,6.250000,6.250000


In [6]:
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])

In [7]:
dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])

In [8]:
params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.2,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=10)

[0]	train-aft-nloglik:6.07962	valid-aft-nloglik:6.11549
[1]	train-aft-nloglik:5.65744	valid-aft-nloglik:5.68985
[2]	train-aft-nloglik:5.27431	valid-aft-nloglik:5.30277
[3]	train-aft-nloglik:4.92694	valid-aft-nloglik:4.95109
[4]	train-aft-nloglik:4.61164	valid-aft-nloglik:4.63256
[5]	train-aft-nloglik:4.32516	valid-aft-nloglik:4.34273
[6]	train-aft-nloglik:4.06503	valid-aft-nloglik:4.07985
[7]	train-aft-nloglik:3.82881	valid-aft-nloglik:3.84084
[8]	train-aft-nloglik:3.61405	valid-aft-nloglik:3.62335
[9]	train-aft-nloglik:3.41886	valid-aft-nloglik:3.42559
[10]	train-aft-nloglik:3.24123	valid-aft-nloglik:3.24596
[11]	train-aft-nloglik:3.07964	valid-aft-nloglik:3.08226
[12]	train-aft-nloglik:2.93247	valid-aft-nloglik:2.93409
[13]	train-aft-nloglik:2.79846	valid-aft-nloglik:2.79841
[14]	train-aft-nloglik:2.67632	valid-aft-nloglik:2.67492
[15]	train-aft-nloglik:2.56493	valid-aft-nloglik:2.56241
[16]	train-aft-nloglik:2.46338	valid-aft-nloglik:2.45940
[17]	train-aft-nloglik:2.37075	valid-aft-

In [9]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

array([4.260553 , 4.6314116, 5.1531754, ..., 2.1848862, 3.77518  ,
       4.382959 ], dtype=float32)

In [10]:
times_true = []
event_true = []
for idx, rows in y_test.iterrows():
    if rows['Survival_label_upper_bound'] == np.inf or not rows['Survival_label_upper_bound']:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = rows['Survival_label_lower_bound']
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [11]:
y_test.head()

,Survival_label_lower_bound,Survival_label_upper_bound
9035,19.250000,inf
8889,18.333333,inf
12207,18.166667,inf
11049,18.833333,inf
8015,1.250000,1.25


In [12]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index

0.8128130413613723